In [4]:
import boto3
import json

Credentials используются из `aws-cli`<br>
https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html

In [5]:
s3 = boto3.resource('s3')

In [6]:
for bucket in s3.buckets.all():
    print(bucket)

s3.Bucket(name='arch1baald-dota')


Notice that we use `.client` instead of `.resource`

In [12]:
s3 = boto3.client('s3')

In [16]:
with open('6227492909-copy.jsonlines', 'wb') as fout:
    s3.download_fileobj('arch1baald-dota', '6227492909.jsonlines', fout)

Проблема подхода в том, что мы скачиваем весь файл и сохраняем его локально на диск.


In [32]:
stream = s3.get_object(Bucket='arch1baald-dota', Key='6227492909.jsonlines')
stream

{'ResponseMetadata': {'RequestId': '2GQ95EF8VV69P5H6',
  'HostId': 'vulzqJx+tSwanT40dm0Rms0CYt4SosEZRN/HbT31E/E9x1hiNsj9IdNkFbZ2g1+vdTrVowAP1HE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'vulzqJx+tSwanT40dm0Rms0CYt4SosEZRN/HbT31E/E9x1hiNsj9IdNkFbZ2g1+vdTrVowAP1HE=',
   'x-amz-request-id': '2GQ95EF8VV69P5H6',
   'date': 'Thu, 23 Jun 2022 20:18:44 GMT',
   'last-modified': 'Thu, 23 Jun 2022 19:46:11 GMT',
   'etag': '"74c98cf5b05f6e568ec7cfd8fe13413e-3"',
   'accept-ranges': 'bytes',
   'content-type': 'application/x-www-form-urlencoded; charset=utf-8',
   'server': 'AmazonS3',
   'content-length': '39239892'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2022, 6, 23, 19, 46, 11, tzinfo=tzutc()),
 'ContentLength': 39239892,
 'ETag': '"74c98cf5b05f6e568ec7cfd8fe13413e-3"',
 'ContentType': 'application/x-www-form-urlencoded; charset=utf-8',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f3670d5d1c0>}

Можно стримить файл по строкам в память

In [35]:
stream = s3.get_object(Bucket='arch1baald-dota', Key='6227492909.jsonlines')
for line in stream['Body'].iter_lines():
    # line is a b'' string
    e = json.loads(line.decode('utf-8'))

In [43]:
json.dumps(e)[:50]

'{"time": -864, "type": "epilogue", "key": "{\\"unkn'

Загрузка в бакет

In [44]:
with open('../replays/6054960498.jsonlines', 'rb') as fin:
    s3.upload_fileobj(fin, 'arch1baald-dota', '6054960498.jsonlines')

Проверка на наличие файла в бакете

In [47]:
from botocore.errorfactory import ClientError

try:
    s3.head_object(Bucket='arch1baald-dota', Key='fakefile.jsonlines')
except ClientError:
    pass

TODO: save resulting files from worker process to s3